In [1]:
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from tqdm import tqdm
from utils.utils import get_readable_file_size

In [2]:
ehr_data_dir = "/home/mengliang/DatasetFolder/mimiciv/2.2"

In [3]:
ehr_cxr_path = os.path.join("database", "mimic_iv_merge.csv")
df_ehr_cxr = pd.read_csv(ehr_cxr_path, index_col=False)
df_ehr_cxr.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,...,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,path,split,report_path
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train,files/p10/p10000032/s50414267.txt
1,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,train,files/p10/p10000032/s50414267.txt
2,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train,files/p10/p10000032/s53189527.txt
3,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect,files/p10/p10000032/s53189527/e084de3b-be89b11...,train,files/p10/p10000032/s53189527.txt
4,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train,files/p10/p10000032/s53911762.txt


In [4]:
print(df_ehr_cxr.columns)
print(df_ehr_cxr["ViewPosition"].value_counts())

print(len(df_ehr_cxr))
print(len(df_ehr_cxr["subject_id"].unique()))
print(len(df_ehr_cxr["study_id"].unique()))

Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'dicom_id', 'PerformedProcedureStepDescription', 'ViewPosition', 'Rows',
       'Columns', 'StudyDate', 'StudyTime',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'path', 'split',
       'report_path'],
      dtype='object')
ViewPosition
AP                147169
PA                 96155
LATERAL            82852
LL                 35129
PA LLD                 4
LAO                    3
RAO                    3
AP AXIAL               2
AP LLD                 2
XTABLE LATERAL         2
AP RLD                 2
SWIMMERS               1
PA RLD                 1
LPO                    1
Name: count, dtype: int64
377095
65379


In [5]:
selected_column = ['subject_id', 'study_id', 'dicom_id', 'PerformedProcedureStepDescription', 
                   'ViewPosition', 'Rows', 'Columns', 'StudyDate', 'StudyTime', 
                   'ViewCodeSequence_CodeMeaning', 'path', 'report_path']

df_cxr_selected_column = df_ehr_cxr[selected_column]

df_cxr_selected_column.loc[:,'StudyDate'] = pd.to_datetime(df_cxr_selected_column.loc[:,'StudyDate'].astype(str), format='%Y%m%d')
# convert study date to string format
df_cxr_selected_column.loc[:,'StudyDate'] = df_cxr_selected_column.loc[:,'StudyDate'].dt.strftime('%Y-%m-%d')
# convert study time to string format
df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:,'StudyTime'].astype(str).str.split('.').str[0].str.zfill(6), format='%H%M%S')
df_cxr_selected_column.loc[:,'StudyTime'] = df_cxr_selected_column.loc[:,'StudyTime'].dt.strftime('%H:%M:%S')
# merge study date and study time

df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:, 'StudyDate'].dt.strftime('%Y-%m-%d') + ' ' + df_cxr_selected_column.loc[:, 'StudyTime'].dt.strftime('%H:%M:%S'))
# convert study time to string format
df_cxr_selected_column.head()

,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,CHEST (PA AND LAT),PA,3056,2544,2180-05-06,2180-05-06 21:30:14,postero-anterior,files/p10/p10000032/s50414267/02aa804e-bde0afd...,files/p10/p10000032/s50414267.txt
1,10000032,50414267,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,CHEST (PA AND LAT),LATERAL,3056,2544,2180-05-06,2180-05-06 21:30:14,lateral,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,files/p10/p10000032/s50414267.txt
2,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,CHEST (PA AND LAT),PA,3056,2544,2180-06-26,2180-06-26 16:55:00,postero-anterior,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,files/p10/p10000032/s53189527.txt
3,10000032,53189527,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,CHEST (PA AND LAT),LATERAL,3056,2544,2180-06-26,2180-06-26 16:55:00,lateral,files/p10/p10000032/s53189527/e084de3b-be89b11...,files/p10/p10000032/s53189527.txt
4,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,CHEST (PORTABLE AP),AP,2705,2539,2180-07-23,2180-07-23 08:05:56,antero-posterior,files/p10/p10000032/s53911762/68b5c4b1-227d048...,files/p10/p10000032/s53911762.txt


In [6]:
icu_admissions = pd.read_csv(os.path.join("outputs", "icu_admissions.csv.gz"), compression="gzip")
icu_admissions.head()

,subject_id,hadm_id,admittime,dischtime,admission_type,admission_location,discharge_location,insurance,marital_status,race
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,ew emer.,emergency room,home,medicaid,widowed,white
1,10000980,26913865,2189-06-27 07:38:00,2189-07-03 03:00:00,ew emer.,emergency room,home health care,medicare,married,black/african american
2,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,ew emer.,emergency room,home health care,other,married,white
3,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,direct emer.,physician referral,home health care,other,married,white
4,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,ew emer.,pacu,home,other,married,white


In [7]:
print(len(icu_admissions["subject_id"].unique()))

50920


In [8]:
# convert time to datetime format
icu_admissions.loc[:, 'admittime'] = pd.to_datetime(icu_admissions.loc[:, 'admittime'])
icu_admissions.loc[:, 'dischtime'] = pd.to_datetime(icu_admissions.loc[:, 'dischtime'])
df_cxr_selected_column.loc[:, 'StudyTime'] = pd.to_datetime(df_cxr_selected_column.loc[:, 'StudyTime'])

# process by subject_id
result_list = []

# enumerate all subject_id
for subject_id, group_B in df_cxr_selected_column.groupby('subject_id'):
    # obtain corresponding admission record
    group_A = icu_admissions[icu_admissions['subject_id'] == subject_id]

    # filter records within the admission time range
    matched = group_B.apply(
        lambda row: group_A.loc[
            (group_A['admittime'] <= row['StudyTime']) & 
            (group_A['dischtime'] >= row['StudyTime']), 
            'hadm_id'
        ].values, 
        axis=1
    )

    # record the matched records
    for index, hadm_ids in enumerate(matched):
        if len(hadm_ids) > 0:
            for hadm_id in hadm_ids:
                result_list.append({
                    'subject_id': subject_id,
                    'hadm_id': hadm_id,
                    #'study_id': group_B.iloc[index]['study_id'],
                    'StudyTime': group_B.iloc[index]['StudyTime'],
                })

# convert to DataFrame
result_df = pd.DataFrame(result_list)
result_df.head()

,subject_id,hadm_id,StudyTime
0,10001217,27703517,2157-12-18 19:16:10
1,10001884,26184834,2131-01-15 04:45:09
2,10001884,26184834,2131-01-12 04:56:56
3,10001884,26184834,2131-01-13 04:49:18
4,10001884,26184834,2131-01-10 12:54:30


In [9]:
result_df.drop_duplicates(inplace=True, keep="first")
print(len(result_df))
print(len(result_df["subject_id"].unique()))
print(len(result_df["hadm_id"].unique()))
#print(len(result_df["study_id"].unique()))

75686
13170
16268


In [11]:
result_df.to_csv('outputs/icu_cxr_subject_info.csv', index=False)

In [12]:
df_icu_cxr = pd.merge(df_cxr_selected_column, result_df, how = 
                      'inner', on=['subject_id', 'StudyTime'])
df_icu_cxr.drop_duplicates(inplace=True, keep='first')
df_icu_cxr.head()

,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path,hadm_id
0,10001217,58913004,5e54fc9c-37c49834-9ac3b915-55811712-9d959d26,CHEST (PORTABLE AP),AP,3050,2539,2157-12-18,2157-12-18 19:16:10,antero-posterior,files/p10/p10001217/s58913004/5e54fc9c-37c4983...,files/p10/p10001217/s58913004.txt,27703517
1,10001884,50376803,469d0d94-3dad5068-efac76ef-a28cc502-68fe6275,CHEST (PORTABLE AP),AP,2539,2517,2131-01-15,2131-01-15 04:45:09,antero-posterior,files/p10/p10001884/s50376803/469d0d94-3dad506...,files/p10/p10001884/s50376803.txt,26184834
2,10001884,50712381,7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9,CHEST (PORTABLE AP),AP,2287,2532,2131-01-12,2131-01-12 04:56:56,antero-posterior,files/p10/p10001884/s50712381/7b25b3ed-e780a52...,files/p10/p10001884/s50712381.txt,26184834
3,10001884,56722923,c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f,CHEST (PORTABLE AP),AP,2539,3050,2131-01-13,2131-01-13 04:49:18,antero-posterior,files/p10/p10001884/s56722923/c1ad3e27-62d05ef...,files/p10/p10001884/s56722923.txt,26184834
4,10001884,57156853,9fd47edd-07087209-b901811e-3e9e5f50-f382f611,CHEST (PORTABLE AP),AP,2964,2539,2131-01-10,2131-01-10 12:54:30,antero-posterior,files/p10/p10001884/s57156853/9fd47edd-0708720...,files/p10/p10001884/s57156853.txt,26184834


In [13]:
print("Patients: ", len(df_icu_cxr["subject_id"].unique()))
print("CXR times: ", len(df_icu_cxr["study_id"].unique()))
print("Admissions: ", len(df_icu_cxr["hadm_id"].unique()))
print("Images: ", len(df_icu_cxr["dicom_id"].unique()))
print("Table length: ", len(df_icu_cxr))

Patients:  13170
CXR times:  73897
Admissions:  16268
Images:  89483
Table length:  89483


In [14]:
df_icu_cxr["path"] = '/home/mengliang/DatasetFolder/mimic-cxr-jpg/2.0.0/' + df_icu_cxr['path']
df_icu_cxr["report_path"] = '/home/mengliang/DatasetFolder/mimic-cxr-jpg/2.0.0/' + df_icu_cxr['report_path']
df_icu_cxr.head()


,subject_id,study_id,dicom_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ViewCodeSequence_CodeMeaning,path,report_path,hadm_id
0,10001217,58913004,5e54fc9c-37c49834-9ac3b915-55811712-9d959d26,CHEST (PORTABLE AP),AP,3050,2539,2157-12-18,2157-12-18 19:16:10,antero-posterior,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,27703517
1,10001884,50376803,469d0d94-3dad5068-efac76ef-a28cc502-68fe6275,CHEST (PORTABLE AP),AP,2539,2517,2131-01-15,2131-01-15 04:45:09,antero-posterior,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,26184834
2,10001884,50712381,7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9,CHEST (PORTABLE AP),AP,2287,2532,2131-01-12,2131-01-12 04:56:56,antero-posterior,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,26184834
3,10001884,56722923,c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f,CHEST (PORTABLE AP),AP,2539,3050,2131-01-13,2131-01-13 04:49:18,antero-posterior,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,26184834
4,10001884,57156853,9fd47edd-07087209-b901811e-3e9e5f50-f382f611,CHEST (PORTABLE AP),AP,2964,2539,2131-01-10,2131-01-10 12:54:30,antero-posterior,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,26184834


In [15]:
print(df_icu_cxr.columns)

Index(['subject_id', 'study_id', 'dicom_id',
       'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns',
       'StudyDate', 'StudyTime', 'ViewCodeSequence_CodeMeaning', 'path',
       'report_path', 'hadm_id'],
      dtype='object')


In [16]:
df_icu_cxr = df_icu_cxr.reindex(columns=['subject_id', 'hadm_id', 'study_id',
                                         'ViewPosition', 'StudyTime', 'path', 
                                         'report_path'])
df_icu_cxr.head()

,subject_id,hadm_id,study_id,ViewPosition,StudyTime,path,report_path
0,10001217,27703517,58913004,AP,2157-12-18 19:16:10,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....
1,10001884,26184834,50376803,AP,2131-01-15 04:45:09,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....
2,10001884,26184834,50712381,AP,2131-01-12 04:56:56,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....
3,10001884,26184834,56722923,AP,2131-01-13 04:49:18,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....
4,10001884,26184834,57156853,AP,2131-01-10 12:54:30,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....,/home/mengliang/DatasetFolder/mimic-cxr-jpg/2....


In [17]:
print(len(df_icu_cxr["subject_id"].unique()))
print(df_icu_cxr["ViewPosition"].value_counts())

13170
ViewPosition
AP                76545
LL                 5886
PA                 4272
LATERAL             575
AP LLD                2
PA LLD                2
XTABLE LATERAL        1
PA RLD                1
AP RLD                1
LPO                   1
Name: count, dtype: int64


In [18]:
df_icu_cxr.to_csv(os.path.join("outputs", "icu_cxr.csv.gz"), 
                               index=False, compression="gzip")

In [19]:
readable_file_size = get_readable_file_size(os.path.join("outputs", "icu_cxr.csv.gz"))
print(f"File size: {readable_file_size}")

File size: 4.35 MB


icu_cxr.csv.gz contains those patients who both have EHR, ICU, and CXR information. We will use this file to filter the patients and admissions for further analysis.

In [22]:
import torch
import ollama

os.environ["CUDA_VISIBLE_DEVICES"] = "5, 6"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
# Open the file in read mode and read the content as a string
with open('files/cxr_description.txt', 'r') as file:
    description_text = file.read()

with open('files/prompt_text.txt', 'r') as file:
    prompt_text = file.read()

# Print the content of the file
print(description_text)
print(prompt_text)

This is the description to the icu_cxr.csv.gz file. This file is located in outputs/icu_cxr.csv.gz.
subject_id: Unique identifier for each patient in the dataset.
hadm_id: Unique identifier for each hospital admission associated with a specific patient.
study_id: Unique identifier for each radiological study (exam), such as a chest X-ray, associated with a specific patient.
ViewPosition: The positioning of the patient when the image was taken (e.g., PA for posterior-anterior, or AP for anterior-posterior).
StudyTime: The time when the radiological study was performed.
path: Path to the image file for the chest X-ray in the dataset.
report_path: Path to the radiology report associated with the chest X-ray. Diagnoses and findings are included in the report.
Please generate python code to answer the question. Only generate code for the question. no explanation and other description. use print to putput the result. the final result variable should be named as result.


In [28]:
# extract information from discharge summary
description_text = description_text
prompt_text = prompt_text
question_text = "what diseases does the patient 10001217 have in the admission 27703517? according to the radiology report?"
input_text = description_text + " " + prompt_text + " " + question_text

response = ollama.chat(model="qwen2.5:14b", messages = [{"role": "user", "content": input_text,}], stream=False)
#response = ollama.chat(model="llama3.1:8b", messages = [{"role": "user", "content": input_text,}], stream=False)
answer_text = response["message"]["content"]
print(answer_text)

```python
import pandas as pd

# Load the dataset
data = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for specific patient (subject_id) and hospital admission (hadm_id)
patient_data = data[(data['subject_id'] == 10001217) & (data['hadm_id'] == 27703517)]

# Extract the report path
report_path = patient_data['report_path'].values[0]

# Read the radiology report from the file
with open(report_path, 'r') as file:
    report_content = file.read()

# Find diseases mentioned in the report
diseases = []  # Assuming we need to manually parse or use NLP techniques here
# For simplicity and without actual parsing logic, let's assume we can directly extract it from content.
# In practice, you might need to implement more complex text processing methods.

# Example of a simple keyword search (not robust but for illustration)
if "pneumonia" in report_content:
    diseases.append("Pneumonia")
if "atelectasis" in report_content:
    diseases.append("Atelectasis")

result = ", ".join(diseases) if d

In [29]:
import pandas as pd

# Load the dataset
data = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for specific patient (subject_id) and hospital admission (hadm_id)
patient_data = data[(data['subject_id'] == 10001217) & (data['hadm_id'] == 27703517)]

# Extract the report path
report_path = patient_data['report_path'].values[0]

# Read the radiology report from the file
with open(report_path, 'r') as file:
    report_content = file.read()

# Find diseases mentioned in the report
diseases = []  # Assuming we need to manually parse or use NLP techniques here
# For simplicity and without actual parsing logic, let's assume we can directly extract it from content.
# In practice, you might need to implement more complex text processing methods.

# Example of a simple keyword search (not robust but for illustration)
if "pneumonia" in report_content:
    diseases.append("Pneumonia")
if "atelectasis" in report_content:
    diseases.append("Atelectasis")

result = ", ".join(diseases) if diseases else "No specific disease mentioned"
print(result)

No specific disease mentioned


In [30]:
# extract information from discharge summary
description_text = description_text
prompt_text = prompt_text
question_text = "Does the patient 10002428 have pleural effusion?"
input_text = description_text + " " + prompt_text + " " + question_text

response = ollama.chat(model="qwen2.5:14b", messages = [{"role": "user", "content": input_text,}], stream=False)
#response = ollama.chat(model="llama3.1:8b", messages = [{"role": "user", "content": input_text,}], stream=False)
answer_text = response["message"]["content"]
print(answer_text)

```python
import pandas as pd

# Load the dataset
df = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for the specific patient and check for pleural effusion in reports
patient_reports = df[df['subject_id'] == 10002428]['report_path']
pleural_effusion = False

for report_path in patient_reports:
    with open(report_path, 'r') as file:
        report_content = file.read()
        if "pleural effusion" in report_content.lower():
            pleural_effusion = True
            break

result = pleural_effusion
print(result)
```


In [31]:
import pandas as pd

# Load the dataset
df = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for the specific patient and check for pleural effusion in reports
patient_reports = df[df['subject_id'] == 10002428]['report_path']
pleural_effusion = False

for report_path in patient_reports:
    with open(report_path, 'r') as file:
        report_content = file.read()
        if "pleural effusion" in report_content.lower():
            pleural_effusion = True
            break

result = pleural_effusion
print(result)

True


In [32]:
# extract information from discharge summary
description_text = description_text
prompt_text = prompt_text
question_text = "Describe the radiology report for the patient 10002428 during its first admission to the hospital."
input_text = description_text + " " + prompt_text + " " + question_text

response = ollama.chat(model="qwen2.5:14b", messages = [{"role": "user", "content": input_text,}], stream=False)
#response = ollama.chat(model="llama3.1:8b", messages = [{"role": "user", "content": input_text,}], stream=False)
answer_text = response["message"]["content"]
print(answer_text)

```python
import pandas as pd

# Load the dataset from the compressed file
df = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for the specific subject_id and hadm_id (assuming hadm_id is unique per admission)
patient_records = df[(df['subject_id'] == 10002428) & (df['hadm_id'] == df[df['subject_id'] == 10002428]['hadm_id'].min())]

# Get the report path for the first admission
report_path = patient_records.iloc[0]['report_path']

# Read and print the content of the radiology report
with open(report_path, 'r') as file:
    report_content = file.read()

result = report_content

print(result)
```


In [33]:
import pandas as pd

# Load the dataset from the compressed file
df = pd.read_csv('outputs/icu_cxr.csv.gz')

# Filter for the specific subject_id and hadm_id (assuming hadm_id is unique per admission)
patient_records = df[(df['subject_id'] == 10002428) & (df['hadm_id'] == df[df['subject_id'] == 10002428]['hadm_id'].min())]

# Get the report path for the first admission
report_path = patient_records.iloc[0]['report_path']

# Read and print the content of the radiology report
with open(report_path, 'r') as file:
    report_content = file.read()

result = report_content

print(result)

                                 FINAL REPORT
 INDICATION:  ___-year-old woman with pseudomonas UTI, intubated for hypoxic
 respiratory failure, assess for interval change.
 
 COMPARISONS:  ___.
 
 Portable semi-upright chest radiograph is presented for review.
 
 Endotracheal tube terminates 2.3 cm above the carina.  Nasogastric and
 Dobbhoff tubes course into the stomach and out of view.  Bilateral right
 greater than left moderate pleural effusions and bibasilar atelectasis are
 unchanged without new pulmonary opacities and unchanged mild pulmonary edema. 
 Left PICC is unchanged.

